In [ ]:
# Import Model (only needs to be ran once)

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import getpass

USER = getpass.getuser()

# Select LLM 0: deepseek, 1: goedel
llm = 1

llm_ids = [
    "deepseek-ai/DeepSeek-Prover-V2-7B",
    "Goedel-LM/Goedel-Prover-V2-8B"
]

llm_id = llm_ids[llm]

tokenizer = AutoTokenizer.from_pretrained(llm_id, cache_dir=f"/work/classtmp/{USER}/models")
model = AutoModelForCausalLM.from_pretrained(llm_id, device_map="auto", cache_dir=f"/work/classtmp/{USER}/models", torch_dtype=torch.bfloat16, trust_remote_code=True)



SyntaxError: invalid syntax (508787569.py, line 17)

In [2]:
# Inference

# This causes the outputs to be deterministic (comment out if this is not desired)
torch.manual_seed(30)

# This prompt was taken from the Deepseek page so it will likely not work with theother LLMs
theorem = """
theorem mathd_algebra_73
  (p q r x : ℂ)
  (h₀ : (x - p) * (x - q) = (r - p) * (r - q))
  (h₁ : x ≠ r) :
  x = p + q - r :=
begin
  sorry
end
"""

formal_statement = f"""
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

{theorem}
""".strip()

prompt = """
Complete the following Lean 4 code:

```lean4
{}
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.
""".strip()

def generate_response(chat, max_new_tokens=8192):
    inputs = tokenizer.apply_chat_template(
        chat,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    output_tokens = model.generate(inputs, max_new_tokens=max_new_tokens)
    output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

    return output_text

prompt = """
The following Lean proof contains issues that need to be resolved. These issues come from the Lean compiler. 
First, write a list of specific instructions that will resolve each one of these issues individually. 
Then, follow the list of instructions to rewrite the proof so it no longer contains the issues provided by the compiler.

Lean compiler feedback:
'simp_all [sub_eq_add_neg]' tactic does nothing.

'ring_nf' tactic does nothing.

'simp [mul_comm, mul_assoc, mul_left_comm]' tactic does nothing.

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat


theorem mathd_algebra_73
  (p q r x : ℂ)
  (h₀ : (x - p) * (x - q) = (r - p) * (r - q))
  (h₁ : x ≠ r) :
  x = p + q - r := by
  have h_main : (x - r) * (x - (p + q - r)) = 0 := by
    have h₂ : (x - p) * (x - q) = (r - p) * (r - q) := h₀
    have h₃ : (x - p) * (x - q) - (r - p) * (r - q) = 0 := by
      rw [h₂]
      ring
    have h₄ : (x - r) * (x - (p + q - r)) = (x - p) * (x - q) - (r - p) * (r - q) := by
      ring_nf
      <;>
      simp [mul_comm, mul_assoc, mul_left_comm]
      <;>
      ring_nf
      <;>
      simp_all [sub_eq_add_neg]
      <;>
      ring_nf
      <;>
      simp_all [sub_eq_add_neg]
      <;>
      ring_nf
      <;>
      linarith
    rw [h₄] at *
    simpa using h₃

  have h_final : x = p + q - r := by
    have h₂ : x - r ≠ 0 := by
      intro h
      apply h₁
      rw [sub_eq_zero] at h
      simpa using h
    have h₃ : x - (p + q - r) = 0 := by
      apply mul_left_cancel₀ h₂
      simpa [sub_mul, mul_sub, mul_comm, mul_assoc, mul_left_comm] using h_main
    have h₄ : x = p + q - r := by
      have h₅ : x - (p + q - r) = 0 := h₃
      have h₆ : x = p + q - r := by
        rw [sub_eq_zero] at h₅
        exact h₅
      exact h₆
    exact h₄

  exact h_final
"""

chat = [
    {"role": "user", "content": prompt.format(formal_statement)},
]

# first response
response = generate_response(chat)
print("Assistant:", response)

followup = "The proof contains unnecessary tactics. Remove them."
chat.append({"role": "assistant", "content": response})
chat.append({"role": "user", "content": followup})

# second response using conversation history
response2 = generate_response(chat)
print("Assistant:", response2)

# chat = [
#   {"role": "user", "content": prompt.format(formal_statement)},
# ]

# inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt", padding=True, truncation=True).to(model.device)

# output_tokens = model.generate(inputs, max_new_tokens=8192)

# output = tokenizer.batch_decode(output_tokens)[0]

# print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Assistant: <｜User｜>
The following Lean proof contains issues that need to be resolved. These issues come from the Lean compiler. 
First, write a list of specific instructions that will resolve each one of these issues individually. 
Then, follow the list of instructions to rewrite the proof so it no longer contains the issues provided by the compiler.

Lean compiler feedback:
'simp_all [sub_eq_add_neg]' tactic does nothing.

'ring_nf' tactic does nothing.

'simp [mul_comm, mul_assoc, mul_left_comm]' tactic does nothing.

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat


theorem mathd_algebra_73
  (p q r x : ℂ)
  (h₀ : (x - p) * (x - q) = (r - p) * (r - q))
  (h₁ : x ≠ r) :
  x = p + q - r := by
  have h_main : (x - r) * (x - (p + q - r)) = 0 := by
    have h₂ : (x - p) * (x - q) = (r - p) * (r - q) := h₀
    have h₃ : (x - p) * (x - q) - (r - p) * (r - q) = 0 := by
      rw [h₂]
      ring
    have h₄ : (x - r) * (x - (p + q - r)) = (x -